In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime

In [ ]:
covid_india_df = pd.read_csv("../input/new-dataset/covid_19_india.csv")
#covid_india_df.head(4)
#covid_india_df.info()
#covid_india_df.describe()
#covid_india_df.count()
covid_india_df.drop(["Sno","Time","ConfirmedIndianNational","ConfirmedForeignNational"], inplace=True, axis=1)
#print('columns succesfully dropped')
covid_india_df['Date']=pd.to_datetime(covid_india_df['Date'],format='%Y-%m-%d')
#covid_india_df.info()
#covid_india_df.head(10)
#Current active cases = confirmed - (cured + deaths)
#covid_india_df['Active_Cases'] = covid_india_df['Confirmed'] - (covid_india_df['Cured'] + covid_india_df['Deaths'])
covid_india_df['ActiveCases'] = covid_india_df['Confirmed'] - (covid_india_df['Cured'] + covid_india_df['Deaths'])
covid_india_df.tail(20)
statewise = pd.pivot_table(covid_india_df,values=["Confirmed","Deaths","Cured"],index="State/UnionTerritory", aggfunc=sum )
statewise.tail(10)
statewise["RecoveryRate"] = statewise["Cured"]*100/statewise["Confirmed"]
statewise["MortalityRate"] = statewise["Deaths"]*100/statewise["Confirmed"]
statewise = statewise.sort_values(by = "Confirmed", ascending=False)
statewise.style.background_gradient(cmap="rainbow")
top_10_active_cases = covid_india_df.groupby(by='State/UnionTerritory').max()[['ActiveCases','Date']].sort_values(by=['ActiveCases'], ascending=False).reset_index()
top_10_active_cases.head(10)
fig = plt.figure(figsize=(30,20))
plt.title("Top 10 Indian States With Currently Active Cases",size=25)
ax=sns.barplot(data=top_10_active_cases.iloc[:10],y = "ActiveCases",x="State/UnionTerritory",linewidth=3,edgecolor='blue')
plt.xlabel("States")
plt.ylabel("Currently Active Cases")
plt.show()



In [ ]:
#Top 10 Indian States Having Highest Mortality
top_10_mortality_cases = covid_india_df.groupby(by='State/UnionTerritory').max()[['Deaths','Date']].sort_values(by=['Deaths'], ascending=False).reset_index()
fig = plt.figure(figsize=(30,20))
plt.title("Top 10 Indian States With Highest Mortality",size=25)
ax=sns.barplot(data=top_10_mortality_cases.iloc[:10],y = "Deaths",x="State/UnionTerritory",linewidth=3,edgecolor='gold')
plt.xlabel("States")
plt.ylabel("Deaths")
plt.show()

In [ ]:
#Growth Trending
fig = plt.figure(figsize = (12,6))

#ax = sns.lineplot(data=covid_india_df[covid_india_df['State/UnionTerritory'].isin(['Maharashtra','Karnataka','Kerala','Tamil Nadu','Uttar Pradesh']),x='Date',y='ActiveCases',hue='State/UnionTerritory])
ax=sns.lineplot(data=covid_india_df[covid_india_df['State/UnionTerritory'].isin(['Maharashtra'])])

In [ ]:
fig = plt.figure(figsize = (12,6))
ax=sns.lineplot(data=covid_india_df[covid_india_df['State/UnionTerritory'].isin(['Maharashtra','Karnataka','Kerala','Tamil Nadu','Uttar Pradesh'])],x='Date',y='ActiveCases',hue='State/UnionTerritory')
ax.set_title('Trending of Active Covid Cases In India By State and Month For Year 2020',size=16)


In [ ]:
vaccine_df = pd.read_csv("../input/covid-vaccine/covid_vaccine_statewise.csv")
#vaccine_df.head(10)
#vaccine_df.info()
#Change Field 'Updated On' to Vaccine_Date 
vaccine_df.rename(columns = {'Updated On': 'Vaccine_Date'}, inplace=True)
#vaccine_df.info()
#how many nulls in this dataset
vaccine_df.isnull().sum()
vaccine = vaccine_df.drop(columns=['Sputnik V (Doses Administered)','AEFI','18-44 Years (Doses Administered)','45-60 Years (Doses Administered)','60+ Years (Doses Administered)'], axis=1)
#vaccine.info()
#PYPLOT pie chart for male and female vaccination rates using plotly library
males = vaccine['Male(Individuals Vaccinated)'].sum()
females = vaccine['Female(Individuals Vaccinated)'].sum()
px.pie(names=['Male','Female'],values=[males,females],title="Male and Female Vaccinations For Year 2020")
#Remove where state = 'India'
vacc = vaccine_df[vaccine_df.State != 'India']
vacc.rename(columns = {'Total Individuals Vaccinated':'Total_Vaxd'},inplace=True)
#Indian States Having Most # Of Vaccinated and Least # Of Vaccinated
#most_vaccinated = vacc.groupby('State')['Total_Vaxd'].sum().to_frame('Total')
most_vaccinated = vacc.groupby('State')['Total_Vaxd'].sum().to_frame('Total_Vaxd')
most_vaccinated.head(10)
most_vaccinated = most_vaccinated.sort_values('Total_Vaxd',ascending=False)[:5]  # Top 5 States
most_vaccinated
#seaborn bar plot of the above chart
fig = plt.figure(figsize = (10,5))
plt.title("Top 5 Vaccinated States India Year 2020", size=20)
x=sns.barplot(data=most_vaccinated.iloc[:10], y=most_vaccinated.Total_Vaxd, x=most_vaccinated.index, linewidth = 3, edgecolor='gold')
plt.xlabel("Indian States")
plt.ylabel("Total Vaccinations Year 2020")
plt.show()
most_vaccinated


